In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
import math
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import *
from shapely.ops import *

In [165]:
videoanal=pd.read_csv('videoanal.csv',sep=';')

In [166]:
videoanal[0:1]

,Дата (начальная остановка),day_type,Прибытие на начальную,Отправление с начальной,Прибытие на конечную,Отправление с конечной,Маршрут,ID трассы (Яндекс)
0,17.08.2020,понедельник,7:11:31,7:11:41,7:46:02,7:49:29,52,2193179888


In [167]:
videoanal['day_type'].unique()

array(['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота',
       'воскресенье'], dtype=object)

In [168]:
videoanal['budny']=0
videoanal['sat']=0

In [169]:
videoanal.loc[(videoanal.day_type == 'понедельник'),'budny']=1
videoanal.loc[(videoanal.day_type == 'вторник'),'budny']=1
videoanal.loc[(videoanal.day_type == 'среда'),'budny']=1
videoanal.loc[(videoanal.day_type == 'четверг'),'budny']=1
videoanal.loc[(videoanal.day_type == 'пятница'),'budny']=1
videoanal.loc[(videoanal.day_type == 'суббота'),'sat']=1
videoanal.loc[(videoanal.day_type == 'воскресенье'),'sat']=1

In [170]:
videoanal['ID трассы (Яндекс)']=videoanal['ID трассы (Яндекс)'].astype('int64')

In [171]:
videoanal=videoanal[['Дата (начальная остановка)','Отправление с начальной','Прибытие на конечную','ID трассы (Яндекс)','budny','sat']]

In [172]:
videoanal[0:1]

,Дата (начальная остановка),Отправление с начальной,Прибытие на конечную,ID трассы (Яндекс),budny,sat
0,17.08.2020,7:11:41,7:46:02,2193179888,1,0


In [175]:
FROM_GTFS=pd.read_csv('FROM_GTFS2.csv',sep=';',encoding='windows-1251')

In [176]:
FROM_GTFS[FROM_GTFS['trip_id']=='0_37881_378810011_10']

,trip_id,arrival_time,stop_id,stop_sequence_x,route_id,service_id,trip_headsign,route_long_name,route_short_name,direction_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,budny,sat,ID трассы (Яндекс)
0,0_37881_378810011_10,16:28:00,143,1,121,0_37881,до Завод Металлург,з-д Металлург - Ж/д вокзал,15,1,1,1,1,1,1,0,0,1.0,0.0,2.193180e+09
1,0_37881_378810011_10,17:29:00,174,260,121,0_37881,до Завод Металлург,з-д Металлург - Ж/д вокзал,15,1,1,1,1,1,1,0,0,1.0,0.0,2.193180e+09


In [177]:
FROM_GTFS=FROM_GTFS[['trip_id','arrival_time','trip_headsign','budny','sat','ID трассы (Яндекс)','stop_sequence_x']]

In [178]:
FROM_GTFS['Прибытие на конечную'] = FROM_GTFS.groupby(['trip_id'])['arrival_time'].shift(-1)

In [179]:
FROM_GTFS[FROM_GTFS['trip_id']=='0_37881_378810011_10']

,trip_id,arrival_time,trip_headsign,budny,sat,ID трассы (Яндекс),stop_sequence_x,Прибытие на конечную
0,0_37881_378810011_10,16:28:00,до Завод Металлург,1.0,0.0,2.193180e+09,1,17:29:00
1,0_37881_378810011_10,17:29:00,до Завод Металлург,1.0,0.0,2.193180e+09,260,NaN


In [180]:
FROM_GTFS['Отправление с начальной'] = FROM_GTFS['arrival_time']

In [181]:
FROM_GTFS=FROM_GTFS[FROM_GTFS['Прибытие на конечную'].notna()]

In [183]:
videoanal_bud=videoanal[videoanal['budny']==1]

In [184]:
len(videoanal_bud)

6088

In [185]:
FROM_GTFS_bud=FROM_GTFS[FROM_GTFS['budny']==1]

In [186]:
len(FROM_GTFS_bud)

4415

In [187]:
res=videoanal_bud.merge(FROM_GTFS_bud, on='ID трассы (Яндекс)', how='left',suffixes=('_video', '_GTFS'))

In [188]:
res['Отправление с начальной_video']=pd.to_datetime(res['Отправление с начальной_video'])
res['Прибытие на конечную_video']=pd.to_datetime(res['Прибытие на конечную_video'])
res['Отправление с начальной_GTFS']=pd.to_datetime(res['Отправление с начальной_GTFS'])
res['Прибытие на конечную_GTFS']=pd.to_datetime(res['Прибытие на конечную_GTFS'])

In [189]:
res['delta1']=res['Отправление с начальной_video'].dt.hour*60+res['Отправление с начальной_video'].dt.minute\
-(res['Отправление с начальной_GTFS'].dt.hour*60+res['Отправление с начальной_GTFS'].dt.minute)

In [190]:
res['delta1']=res['delta1']

In [191]:
res['delta2']=res['Прибытие на конечную_video'].dt.hour*60+res['Прибытие на конечную_video'].dt.minute\
-(res['Прибытие на конечную_GTFS'].dt.hour*60+res['Прибытие на конечную_GTFS'].dt.minute)
res['delta2']=res['delta2']

In [192]:
res=res[res['delta1']>=0]

In [193]:
res=res[res['Отправление с начальной_GTFS'].notna()]

In [194]:
res = res.sort_values(by=['Дата (начальная остановка)','ID трассы (Яндекс)','Отправление с начальной_video','delta1','delta2'])

In [195]:
res=res.groupby(['Дата (начальная остановка)','ID трассы (Яндекс)','Отправление с начальной_video','Прибытие на конечную_video']).first().reset_index()

In [196]:
len(res)

4115

In [2]:
routes=pd.read_csv('routes.csv')

In [3]:
res[0:1]

NameError: name 'res' is not defined

In [4]:
routes

,route_number,type_ts,route_long_name,trip_id
0,1,tramway,Автостанция Аврора_Пионерская улица,2193179787
1,76,bus,113-й километр_Школа / Преображенская улица,3979942320
2,76,bus,113-й километр_Школа / Преображенская улица,3979933560
3,36,bus,Хлебная площадь_Совхоз Рубёжное,2193179236
4,36,bus,Хлебная площадь_Совхоз Рубёжное,3121462497
...,...,...,...,...
1751,193,bus,Могутовая_Берёзовый Солонец,3266429687
1752,Красный Яр — Чубовка,minibus,Автостанция Красный Яр_Чубовка,3281636077
1753,Красный Яр — Чубовка,minibus,Автостанция Красный Яр_Чубовка,3697987324
1754,Красный Яр — Чубовка,minibus,Автостанция Красный Яр_Чубовка,3697986624


In [200]:
routes=routes[['route_number','type_ts','route_long_name','trip_id']]

In [201]:
routes['trip_id']=routes['trip_id'].astype('int64')

In [202]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Дата (начальная остановка)     4115 non-null   object        
 1   ID трассы (Яндекс)             4115 non-null   int64         
 2   Отправление с начальной_video  4115 non-null   datetime64[ns]
 3   Прибытие на конечную_video     4115 non-null   datetime64[ns]
 4   budny_video                    4115 non-null   int64         
 5   sat_video                      4115 non-null   int64         
 6   trip_id                        4115 non-null   object        
 7   arrival_time                   4115 non-null   object        
 8   trip_headsign                  4115 non-null   object        
 9   budny_GTFS                     4115 non-null   float64       
 10  sat_GTFS                       3989 non-null   float64       
 11  stop_sequence_x  

In [203]:
res2=res.merge(routes, left_on='ID трассы (Яндекс)', right_on='trip_id', how='left')

In [204]:
res2.to_csv('result_bud2.csv', index=False, encoding='windows-1251')

In [78]:
res2

,Дата (начальная остановка),ID трассы (Яндекс),Отправление с начальной_video,Прибытие на конечную_video,budny_video,sat_video,trip_id_x,arrival_time,trip_headsign,budny_GTFS,sat_GTFS,stop_sequence_x,Прибытие на конечную_GTFS,Отправление с начальной_GTFS,delta1,delta2,route_number,type_ts,route_long_name,trip_id_y
0,01.11.2020,2193179186,2020-11-19 13:19:53,2020-11-19 13:44:51,0,1,0_37900_379002021_12,13:16:00,до Троллейбусное депо № 2,0.0,1.0,1.0,2020-11-19 13:41:00,2020-11-19 13:16:00,3.0,3.0,7,trolleybus,Троллейбусное депо № 2_Завод Металлург,2.193179e+09
1,01.11.2020,2193179658,2020-11-19 07:18:39,2020-11-19 08:01:40,0,1,0_38702_387022081_3,7:18:00,до Постников овраг,0.0,1.0,1.0,2020-11-19 08:01:00,2020-11-19 07:18:00,0.0,0.0,13,tramway,Костромской переулок_Постников овраг,2.193180e+09
2,01.11.2020,2193179658,2020-11-19 08:59:07,2020-11-19 09:37:33,0,1,0_38702_387022081_5,8:56:00,до Постников овраг,0.0,1.0,1.0,2020-11-19 09:39:00,2020-11-19 08:56:00,3.0,-2.0,13,tramway,Костромской переулок_Постников овраг,2.193180e+09
3,01.11.2020,2193179658,2020-11-19 11:03:17,2020-11-19 11:43:22,0,1,0_38702_387022081_7,11:03:00,до Постников овраг,0.0,1.0,1.0,2020-11-19 11:46:00,2020-11-19 11:03:00,0.0,-3.0,13,tramway,Костромской переулок_Постников овраг,2.193180e+09
4,01.11.2020,2193179658,2020-11-19 12:39:55,2020-11-19 13:19:54,0,1,0_38702_387022031_9,12:31:00,до Постников овраг,0.0,1.0,1.0,2020-11-19 13:14:00,2020-11-19 12:31:00,8.0,5.0,13,tramway,Костромской переулок_Постников овраг,2.193180e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,31.10.2020,2193180128,2020-11-19 15:45:40,2020-11-19 16:15:00,0,1,0_37996_379962032_4,15:44:00,до Постников овраг,0.0,1.0,1.0,2020-11-19 16:20:00,2020-11-19 15:44:00,1.0,-5.0,18,tramway,Постников овраг_Автостанция Аврора,2.193180e+09
1833,31.10.2020,2193180128,2020-11-19 17:24:16,2020-11-19 17:58:09,0,1,0_37996_379962032_7,17:23:00,до Постников овраг,0.0,1.0,1.0,2020-11-19 17:59:00,2020-11-19 17:23:00,1.0,-1.0,18,tramway,Постников овраг_Автостанция Аврора,2.193180e+09
1834,31.10.2020,2193180128,2020-11-19 18:45:31,2020-11-19 19:18:41,0,1,0_37996_379962022_7,18:26:00,до Постников овраг,0.0,1.0,1.0,2020-11-19 19:02:00,2020-11-19 18:26:00,19.0,16.0,18,tramway,Постников овраг_Автостанция Аврора,2.193180e+09
1835,31.10.2020,2193180144,2020-11-19 07:39:58,2020-11-19 09:07:02,0,1,0_40931_409312011_4,7:38:00,до Автостанция Аврора,0.0,1.0,1.0,2020-11-19 07:51:00,2020-11-19 07:38:00,1.0,76.0,1,tramway,Автостанция Аврора_Пионерская улица,2.193180e+09


In [52]:
stop_times=pd.read_csv('stop_times.txt')

In [53]:
trips=pd.read_csv('trips.txt')

In [54]:
route_id=pd.read_csv('routes.txt')

In [55]:
trips=trips.merge(route_id,how='left',on='route_id')

In [9]:
trips_dop=trips[trips['route_short_name'].isin(['24','22','1','17'])&(trips['route_type']==3)]\
.append(trips[trips['route_short_name'].isin(['15','9','21','11','19','25','5','20','12'])&(trips['route_type']==0)])\
.append(trips[trips['route_short_name'].isin(['12','17','19',])&(trips['route_type']==5)])

In [15]:
trips_id=pd.read_csv('trips_dop.csv',sep=';')

In [56]:
trips

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,agency_id,route_short_name,route_long_name,route_type
0,121,0_37881,0_37881_378810011_1,до Железнодорожный вокзал,0,0_37881_E,1,15,з-д Металлург - Ж/д вокзал,5
1,121,0_37881,0_37881_378810011_10,до Завод Металлург,1,0_37881_B,1,15,з-д Металлург - Ж/д вокзал,5
2,121,0_37881,0_37881_378810011_11,до Железнодорожный вокзал,0,0_37881_A,1,15,з-д Металлург - Ж/д вокзал,5
3,121,0_37881,0_37881_378810011_12,до Завод Металлург,1,0_37881_B,1,15,з-д Металлург - Ж/д вокзал,5
4,121,0_37881,0_37881_378810011_13,до Железнодорожный вокзал,0,0_37881_A,1,15,з-д Металлург - Ж/д вокзал,5
...,...,...,...,...,...,...,...,...,...,...
14935,176,0_40980,0_40980_409809041_5,до Костромской переулок,1,0_40980_B,1,24к,Костромской пер.-15-й микрорайон,0
14936,176,0_40980,0_40980_409809041_6,до 15-й микрорайон,0,0_40980_A,1,24к,Костромской пер.-15-й микрорайон,0
14937,176,0_40980,0_40980_409809041_7,до Юнгородок,1,0_40980_F,1,24к,Костромской пер.-15-й микрорайон,0
14938,176,0_40980,0_40980_409809041_8,до Костромской переулок,1,0_40980_B,1,24к,Костромской пер.-15-й микрорайон,0


In [125]:
#stop_times_dop=stop_times[stop_times['trip_id'].isin(trips_dop['trip_id'].unique())]

In [57]:
stop_times_dop=stop_times

In [58]:
stop_times_dop=stop_times_dop.sort_values(by=['trip_id','arrival_time'])

In [59]:
stop_times_dop

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,0_37554_375541031_1,05:56:00,05:56:00,562,4
1,0_37554_375541031_1,06:00:00,06:00:00,28,5
2,0_37554_375541031_1,06:03:00,06:03:00,29,6
3,0_37554_375541031_1,06:04:00,06:04:00,30,7
4,0_37554_375541031_1,06:06:00,06:06:00,31,8
...,...,...,...,...,...
421881,0_40980_409809041_9,15:52:00,15:52:00,1302,114
421882,0_40980_409809041_9,15:53:00,15:53:00,1308,115
421883,0_40980_409809041_9,15:55:00,15:55:00,1197,116
421884,0_40980_409809041_9,15:56:00,15:56:00,1195,117


In [60]:
stop_times_dop_first=stop_times_dop.groupby('trip_id').first().reset_index()

In [61]:
stop_times_dop_last=stop_times_dop.groupby('trip_id').last().reset_index()

In [62]:
stop_times_dop_first['stop_sequence']=1

In [63]:
stop_times_res=stop_times_dop_first.append(stop_times_dop_last)

In [64]:
stop_times_res

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,0_37554_375541031_1,05:56:00,05:56:00,562,1
1,0_37554_375541031_2,06:29:00,06:29:00,143,1
2,0_37554_375541031_3,07:43:00,07:43:00,466,1
3,0_37554_375541031_4,08:55:00,08:55:00,143,1
4,0_37554_375541031_5,10:32:00,10:32:00,466,1
...,...,...,...,...,...
17615,0_40980_409809041_5,08:39:00,08:39:00,1283,61
17616,0_40980_409809041_6,09:01:00,09:01:00,1193,72
17617,0_40980_409809041_7,09:34:00,09:34:00,99,87
17618,0_40980_409809041_8,15:37:00,15:37:00,1283,107


In [65]:
trips

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,agency_id,route_short_name,route_long_name,route_type
0,121,0_37881,0_37881_378810011_1,до Железнодорожный вокзал,0,0_37881_E,1,15,з-д Металлург - Ж/д вокзал,5
1,121,0_37881,0_37881_378810011_10,до Завод Металлург,1,0_37881_B,1,15,з-д Металлург - Ж/д вокзал,5
2,121,0_37881,0_37881_378810011_11,до Железнодорожный вокзал,0,0_37881_A,1,15,з-д Металлург - Ж/д вокзал,5
3,121,0_37881,0_37881_378810011_12,до Завод Металлург,1,0_37881_B,1,15,з-д Металлург - Ж/д вокзал,5
4,121,0_37881,0_37881_378810011_13,до Железнодорожный вокзал,0,0_37881_A,1,15,з-д Металлург - Ж/д вокзал,5
...,...,...,...,...,...,...,...,...,...,...
14935,176,0_40980,0_40980_409809041_5,до Костромской переулок,1,0_40980_B,1,24к,Костромской пер.-15-й микрорайон,0
14936,176,0_40980,0_40980_409809041_6,до 15-й микрорайон,0,0_40980_A,1,24к,Костромской пер.-15-й микрорайон,0
14937,176,0_40980,0_40980_409809041_7,до Юнгородок,1,0_40980_F,1,24к,Костромской пер.-15-й микрорайон,0
14938,176,0_40980,0_40980_409809041_8,до Костромской переулок,1,0_40980_B,1,24к,Костромской пер.-15-й микрорайон,0


In [24]:
#stop_times_res=stop_times_res.merge(trips_id,on='trip_id',how='left')

In [66]:
stop_times_res=stop_times_res.merge(trips,on='trip_id',how='left')

In [68]:
stop_times_res[stop_times_res['route_id'].notna()]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,service_id,trip_headsign,direction_id,shape_id,agency_id,route_short_name,route_long_name,route_type
166,0_37881_378810011_1,07:10:00,07:10:00,181,1,121.0,0_37881,до Железнодорожный вокзал,0.0,0_37881_E,1.0,15,з-д Металлург - Ж/д вокзал,5.0
167,0_37881_378810011_10,16:28:00,16:28:00,143,1,121.0,0_37881,до Завод Металлург,1.0,0_37881_B,1.0,15,з-д Металлург - Ж/д вокзал,5.0
168,0_37881_378810011_11,18:14:00,18:14:00,174,1,121.0,0_37881,до Железнодорожный вокзал,0.0,0_37881_A,1.0,15,з-д Металлург - Ж/д вокзал,5.0
169,0_37881_378810011_12,19:18:00,19:18:00,143,1,121.0,0_37881,до Завод Металлург,1.0,0_37881_B,1.0,15,з-д Металлург - Ж/д вокзал,5.0
170,0_37881_378810011_13,20:26:00,20:26:00,174,1,121.0,0_37881,до Железнодорожный вокзал,0.0,0_37881_A,1.0,15,з-д Металлург - Ж/д вокзал,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35235,0_40980_409809041_5,08:39:00,08:39:00,1283,61,176.0,0_40980,до Костромской переулок,1.0,0_40980_B,1.0,24к,Костромской пер.-15-й микрорайон,0.0
35236,0_40980_409809041_6,09:01:00,09:01:00,1193,72,176.0,0_40980,до 15-й микрорайон,0.0,0_40980_A,1.0,24к,Костромской пер.-15-й микрорайон,0.0
35237,0_40980_409809041_7,09:34:00,09:34:00,99,87,176.0,0_40980,до Юнгородок,1.0,0_40980_F,1.0,24к,Костромской пер.-15-й микрорайон,0.0
35238,0_40980_409809041_8,15:37:00,15:37:00,1283,107,176.0,0_40980,до Костромской переулок,1.0,0_40980_B,1.0,24к,Костромской пер.-15-й микрорайон,0.0


In [69]:
service=pd.read_csv('calendar.txt')

In [70]:
service

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,0_37554,1,1,1,1,1,0,0,20141201,20211014
1,0_37881,1,1,1,1,1,0,0,20141201,20211014
2,0_37900,0,0,0,0,0,1,1,20140522,20211014
3,0_37901,1,1,1,1,1,0,0,20140702,20211014
4,0_37902,1,1,1,1,1,0,0,20140701,20211014
...,...,...,...,...,...,...,...,...,...,...
183,0_40956,1,1,1,1,1,0,0,20200723,20211014
184,0_40972,1,1,1,1,1,1,1,20200902,20211014
185,0_40973,1,1,1,1,1,1,1,20200902,20211014
186,0_40979,1,1,1,1,1,0,0,20190912,20211014


In [71]:
stop_times_res=stop_times_res.merge(service,on='service_id',how='left')

In [72]:
stop_times_res[0:1]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,service_id,trip_headsign,direction_id,shape_id,...,route_type,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,0_37554_375541031_1,05:56:00,05:56:00,562,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
stop_times_res=stop_times_res.rename(columns={'ID трассы (яндекс)':'ID трассы (Яндекс)','stop_sequence':'stop_sequence_x'})

In [31]:
stop_times_res=stop_times_res[['trip_id','arrival_time','stop_id','stop_sequence_x','route_id','service_id','trip_headsign','route_long_name','route_short_name','direction_id','monday','tuesday','wednesday','thursday','friday','saturday','sunday','ID трассы (Яндекс)']]

In [160]:
stop_times_res1=pd.read_csv('FROM_GTFS.csv',sep=';')

In [161]:
stop_times_res1=stop_times_res1

In [162]:
stop_times_res1=stop_times_res1.append(stop_times_res)

In [74]:
stop_times_res.to_csv('FROM_GTFS3.csv')

In [73]:
len(stop_times_res)

35240